In [3]:
import pandas as pd

In [4]:
df_gc = pd.read_csv('../data/usga_scrdb_aug_2024.csv')

## Calc Score Differentials

### One at a time

In [ ]:
def calc_score_diff(adjusted_gross_score, course_rating, slope_rating, pcc=0):
    """
    Calculate the score differential for a round of golf.

    Parameters:
    adjusted_gross_score (float): The adjusted gross score for the round.
    course_rating (float): The course rating.
    slope_rating (int): The slope rating.
    pcc (int, optional): The Playing Conditions Calculation adjustment. Default is 0.

    Returns:
    float: The score differential.
    """
    score_differential = (113 / slope_rating) * (adjusted_gross_score - course_rating - pcc)

    return score_differential

In [ ]:
# Example usage:
adjusted_gross_score = 85
course_rating = 72.0
slope_rating = 130
pcc = 0  # Adjust if there's a PCC adjustment

differential = calc_score_diff(adjusted_gross_score, course_rating, slope_rating, pcc)
print(f"Score Differential: {differential:.2f}")

### Multiple at once

In [ ]:
def calc_handicap_index(adjusted_gross_scores, course_ratings, slope_ratings, pcc_adjustments=None):
    """
    Calculate the USGA Handicap Index for a player.

    Parameters:
    adjusted_gross_scores (list of float): List of adjusted gross scores for each round.
    course_ratings (list of float): List of course ratings corresponding to each round.
    slope_ratings (list of int): List of slope ratings corresponding to each round.
    pcc_adjustments (list of int, optional): List of PCC adjustments for each round. Default is None.

    Returns:
    float: The player's Handicap Index.
    """
    if pcc_adjustments is None:
        pcc_adjustments = [0] * len(adjusted_gross_scores)

    score_differentials = []
    for i in range(len(adjusted_gross_scores)):
        differential = (113 / slope_ratings[i]) * (adjusted_gross_scores[i] - course_ratings[i] - pcc_adjustments[i])
        score_differentials.append(differential)

    # Sort the score differentials and select the lowest 8 if there are at least 20 rounds
    score_differentials.sort()
    num_differentials = min(8, len(score_differentials))
    best_differentials = score_differentials[:num_differentials]

    # Calculate the average of the selected differentials and apply the 0.96 multiplier
    if best_differentials:
        average_differential = sum(best_differentials) / num_differentials
        handicap_index = average_differential * 0.96
    else:
        handicap_index = None

    return handicap_index

In [ ]:
# Example usage:
adjusted_gross_scores = [85, 90, 88, 92, 87, 86, 84, 89, 91, 85]
course_ratings = [72.0, 71.5, 72.3, 73.0, 71.8, 72.5, 72.0, 71.9, 72.2, 72.4]
slope_ratings = [130, 128, 132, 135, 129, 131, 130, 127, 133, 134]
pcc_adjustments = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  # Adjust if needed

handicap_index = calc_handicap_index(adjusted_gross_scores, course_ratings, slope_ratings, pcc_adjustments)
print(f"Handicap Index: {handicap_index:.2f}")

### Take in Scores, Course info and Return Handicap

In [17]:
def calculate_handicap_index_from_courses(df_gc, scores, course_info):
    """
    Calculate the USGA Handicap Index for a player based on course information.

    Parameters:
    df_gc (pd.DataFrame): DataFrame containing course information.
    scores (list of float): List of adjusted gross scores for each round.
    course_info (list of tuple): List of tuples containing course name, city, state, tee name, and gender.

    Returns:
    float: The player's Handicap Index.
    """
    score_differentials = []

    for i, (course_name, city, state, tee_name, gender) in enumerate(course_info):
        # Look up course rating and slope rating from the DataFrame
        course_data = df_gc[(df_gc['Course Name'] == course_name) &
                            (df_gc['City'] == city) &
                            (df_gc['State'] == state) &
                            (df_gc['Tee Name'] == tee_name) &
                            (df_gc['Gender'] == gender) &
                            (df_gc['Back (9)'] != '/')]

        if not course_data.empty:
            course_rating = course_data.iloc[0]['Course Rating™']
            slope_rating = course_data.iloc[0]['Slope Rating®']

            # Calculate score differential
            differential = (113 / slope_rating) * (scores[i] - course_rating)
            score_differentials.append(differential)
        else:
            print(f"Course information not found for: {course_name}, {city}, {state}, {tee_name}")

    # Sort the score differentials and select the lowest 8 if there are at least 20 rounds
    score_differentials.sort()
    num_differentials = min(8, len(score_differentials))
    best_differentials = score_differentials[:num_differentials]

    # Calculate the average of the selected differentials and apply the 0.96 multiplier
    if best_differentials:
        average_differential = sum(best_differentials) / num_differentials
        handicap_index = average_differential * 0.96
    else:
        handicap_index = None

    return handicap_index

In [18]:
# Example usage:
# Assuming df_gc is your DataFrame with course data
scores = [85, 90, 88, 92, 87, 86, 84, 89, 91, 85]
course_info = [
    ("Colorado Golf Club", "Parker", "CO", "Championship", "M"),
    ("Colorado Golf Club", "Parker", "CO", "Champ/Black", "M"),
    ("Colorado Golf Club", "Parker", "CO", "Black", "M"),
    ("Colorado Golf Club", "Parker", "CO", "Black/Paintbrush", "M"),
    ("Colorado Golf Club", "Parker", "CO", "Paintbrush", "M"),
    ("Colorado Golf Club", "Parker", "CO", "Championship", "M"),
    ("Colorado Golf Club", "Parker", "CO", "Champ/Black", "M"),
    ("Colorado Golf Club", "Parker", "CO", "Black", "M"),
    ("Colorado Golf Club", "Parker", "CO", "Black/Paintbrush", "M"),
    ("Colorado Golf Club", "Parker", "CO", "Paintbrush", "M")
]

handicap_index = calculate_handicap_index_from_courses(df_gc, scores, course_info)
print(f"Handicap Index: {handicap_index:.2f}")

Handicap Index: 8.56


In [23]:
scores = [100,92,95,100,88,94,86]
course_info = [
    ("Bear Creek Golf Club", "Denver", "CO", "Gold", "M"),
    ("Bear Dance Golf Club", "Larkspur", "CO", "Blue/Black", "M"),
    ("Colorado Golf Club", "Parker", "CO", "Paintbrush", "M"),
    ("Colorado Golf Club", "Parker", "CO", "Paintbrush", "M"),
    ("Applewood Golf Course", "Golden", "CO", "White", "M"),
    ("Banff Springs", "Banff", "AB", "Black/White", "M"),
    ("Applewood Golf Course", "Golden", "CO", "Blue", "M")
]

handicap_index = calculate_handicap_index_from_courses(df_gc, scores, course_info)
print(f"Handicap Index: {handicap_index:.2f}")

Handicap Index: 18.53
